# Word2Vecの紹介
本紹介は，ツールの歴史的背景や詳細な技術的・構成的情報は簡素化し，ツール利用者向けのチュートリアルを指向しています．ご了承ください．

## Word2Vecとは
Word2Vecは，単語をベクトル空間にマッピングして活用する技術の一つです．言い換えれば，単語の意味をベクトル空間上の数値として表そうとした技術とも言えます．「各単語は周辺語に関連する」というアイディアで機械学習技術を応用しており，様々な文章による学習結果として類似性や関連性がある単語がベクトル空間上の近しい位置に配置されることになります．

## Word2Vec入門

ここでは，自然言語処理ライブラリである``gensim``によるword2vecの実装を利用する例を取り上げます．まずgensimをインストールするところから始めます．

In [ ]:
# gensimのインストール
!pip install gensim

### モデルの学習や保存の実行
次のプログラム例は，Word2Vecによる学習や，その結果として得られるモデル，その保存について処理するものです．
Word2Vec(sentences=...)コンストラクタの実行により学習が行なわれます．このときの引数の種類は数多くのものがあります．完全なものとしては https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec を参照してください．

本例に示していない引数としては，たとえば，``sg``は，値が1の場合はskip-gramで学習し，0の場合はCBOWで学習することになります．skip-gramは中心語から周辺語を予測するように学習し，反対にCBOWは周辺語から中心語を予測するように学習します．（中心語は周辺語と対照的な呼び名であり，必ずしも文の中心にある語を表す訳ではありません．）

また，``hs``は，1とするとhierarchical softmax，0とするとnegative samplingといった学習方法を選択できます．negative samplingを用いる場合は，``negative``でノイズとなる語をどの程度扱うかを（通例5-20の範囲で）指定したりします．

このような引数調整は深層学習を使いこなす上で必要な知識ですが，Word2VecはBERTやLLMに比べれば古い技術であり，多くの文献がありますので本資料では技術的な説明は割愛し，基本的な用例を紹介するに留めます．



In [ ]:
from gensim.test.utils import common_texts #お試し用の学習データをgensimが用意している．
from gensim.models import Word2Vec #Word2Vecをインポート

# Word2Vecで学習している．引数のsentencesに学習データを与えます．
# vector_sizeはベクトルの次元数を表します．
# windowは，予測したい語から周辺語までの最大距離，この数値が高いほど多くの周辺語を扱うようになる．
# min_countは，学習に用いる語を決めるための語の頻出度への閾値，この閾値以上に登場した語を学習に用いる．
# workersは，学習に用いるスレッド数，リソースがあれば，workerが多いほど早めに処理が終わる可能性がある．
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

ちなみに上記のプログラム例では，common_textsの中身は``[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system' ...``のような内容で，文ごとに単語が要素として格納された二次元のリストになっています．

大規模なコーパス（学習用に整理した文書データ）であれば，学習に時間がかかりますので，学習結果となるモデルをmodel.saveにより保存します．一度保存してしまえば，下記プログラムのようにloadすればモデルを再利用できます．
既存のモデルに対して，追加の学習をしたければ，trainメソッドを利用するとできます．
trainメソッドのAPI仕様の全貌は https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.train に記載があります．


In [ ]:
model = Word2Vec.load("word2vec.model")
# 第一引数は，追加で学習するデータ
# total_examplesは学習対象データの文の数，例では["hello", "world"]という1文のみなので1
# epochsは世代．学習対象データを何度繰り返して学習するか．なお，コンストラクタで学習したときのdpochsは既定値の5となっていた．
model.train([["hello", "world"]], total_examples=1, epochs=1)

出力結果について補足します．
> WARNING:gensim.models.word2vec:Effective 'alpha' higher than previous training cycles

これは前回の学習時の``alpha``パラメタを前回のトレーニングサイクルよりも高いことを示しています．``alpha``は学習率を表しますが，モデルを収束するためには，学習率がepochが進むにつれて，小さくなるようにすることが良いとされています．学習率を下げると学習結果がモデルに反映されにくくなります．なお，Word2Vecコンストラクタでの学習時では引数で調整していませんので，既定のalpha値は0.025でmin_alpha値は0.001となります．

> (0, 2)

これは，学習した語の数 と 与えられた語の数 の組を表しています．0は学習された単語は0を表しており，モデルに影響を与えていないことがわかります．2は与えた単語数*epoch数になり，ここでepochsは1でしたので，2語となっています．（本例は導入説明のためのプログラムですので効果は気にしていません．）

### 単語ベクトル
学習したモデルを作成したら，そこから単語のベクトル表現を取得できます．（単語が学習時にモデルに反映されていなければエラーになります．たとえば，'hello'を追加学習したつもりでも学習された単語は0になっていたので，ベクトルは取得できません．）

単語のベクトル表現は，``model.wv['ベクトルを取得したい単語']``で得られます．また，``model.wv.most_similar('ベクトルを取得したい単語', topn=10)``で上位10(topnの値)個の類似語を取得できます．類似語の計算は既定ではベクトル間のcos類似度で計算されており，結果の数値が高かったものを似ているとしています．（学習が不十分であれば，当然結果の妥当性も低い可能性があります．）

In [ ]:
vector = model.wv['computer']  # 単語'computer'のベクトル表現を取得
sims = model.wv.most_similar('computer', topn=10)  # 単語'computer'に類似する単語上位10個を取得

In [ ]:
vector

In [ ]:
sims

次に学習で得られたオブジェクトの容量を小さくし，読み出しを高速化する方法について説明します．これまでのWord2Vecオブジェクトはtrainメソッドの呼び出しにより追加学習が可能でしたが，追加学習できるようにデータを揃えていたとも言えます．そのため，loadメソッドは時間がかかりやすく，loadしたオブジェクトも比較的大きいものになります．そこで，学習で得られたベクトル（とその単語）のみを保存することで，追加学習等はできなくなりますが容量を小さくし，利用までを高速化することができます．

In [ ]:
from gensim.models import KeyedVectors

# 単語とそのベクトルのみを保存するために，modelにあるwvのみを保存する．
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# 単語とそのベクトルのみをread-onlyで読み込む（プロセス間で共有可能）
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

sims = wv.most_similar('computer', topn=10) # 単語'computer'に類似する単語上位10個を取得

sims

### 実践
今回は@singletongue氏がGithubでWikiEntVecプロジェクトとして公開している訓練済みモデルを利用します．容量的に一番小さい100次元のモデルを選んでいます．https://github.com/singletongue/WikiEntVec/releases

In [ ]:
!wget 'https://github.com/singletongue/WikiEntVec/releases/download/20190520/jawiki.word_vectors.100d.txt.bz2' -O 'jawiki.word_vectors.100d.txt.bz2'

In [ ]:
# jawiki.word_vectors.100d.txt.bz2を展開すると，word2vec形式のテキストファイルが入っているのでload_word2vec_formatメソッドで読み込み（T4エンジンで2分弱ほどかかります．）
wv = KeyedVectors.load_word2vec_format("jawiki.word_vectors.100d.txt.bz2")

In [ ]:
# 上記セルのloadの処理が時間がかかるので，別セルで続きを書きます．
sims = wv.most_similar('computer', topn=10) # 単語'computer'に類似する単語上位10個を取得
sims

上記プログラム例の実行結果は，冒頭のお試し用データに比べて``computer``のより
類似した語がランキングされていることがわかります．

In [ ]:
sims = wv.most_similar('信州大学', topn=10)
sims

日本語の単語でも類似する語がランキングされているように見えます．ただし，未学習の語（例'信州大学工学部'）の類語を取得しようとするとエラーになりますので注意してください．

## 手持ちの文書で学習
ここでは，手持ちの文章で学習する方法を学びます．今回は処理速度を考慮して既に紹介しているMeCab (fugashi)を用いた方法を紹介します．

In [ ]:
# UniDic辞書を用いるfugashiのインストール
!pip install fugashi[unidic]
# 辞書(500MB程度)のダウンロード
!python -m unidic download
# 長文テキストの解析をデモする文章例
!wget 'https://www.aozora.gr.jp/cards/000148/files/773_14560.html' -O 'bunko.html' #こころ（青空文庫）
#ストップワードとして公開されているテキストを取得
!wget 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt' -O 'stopword.txt'

以下のプログラムでは名詞に限定して学習をさせますので，得られたモデルでの類語検索も名詞でしかできません．また，今回は句点と空白で分割します．周辺語を適切に学習させるためには，適切に一文一文を抽出させることが望ましいですが，この処理は文章に応じた処理になりますので，ここでは扱いません．

In [ ]:
from bs4 import BeautifulSoup #マークアップ言語による記述からデータを抽出するためのライブラリ
from collections import Counter
from fugashi import Tagger
import re # 正規表現のライブラリ
from gensim.models import Word2Vec #Word2Vecをインポート

#単なるtxtファイルに書かれた文章を扱う場合はBeautifulSoupは不要です．
#そのtxtファイルをopenして文字列を取り出すのみで十分でしょう．
soup = BeautifulSoup(open("bunko.html", encoding="shift_jis"))
text = soup.find("div", "main_text").text #main_text classのdivタグに本文がある．
#ストップワードの読み込み，改行区切りでの単語分割
stopwords = open("stopword.txt", 'r').read().split('\n')
#カウントする形態素の品詞指定
allowed_pos = ["名詞"]

#分割しても改行コードなどは残る．名詞を取り上げるとして今回は不問にする．
sentences = re.split('。| | ', text)

tagger = Tagger()
sentence_words = []
for sentence in sentences :
    words = []
    for word in tagger(sentence) :
        if word.feature.orthBase not in stopwords and word.feature.pos1 in allowed_pos:
              words.append(str(word.feature.orthBase))
    if words :
        sentence_words.append(words)

# モデルの生成（学習データとなる単語数が少ないためepochsを100にしているが，データによっては過学習に注意する必要がある．）
model = Word2Vec(sentences=sentence_words, vector_size=100, window=5, min_count=1, workers=4, epochs=100)
sentence_words

In [ ]:
model.wv.most_similar("先生", topn=10)

## 演習
1. 手持ちの文書に対し，Word2Vecを利用し，任意の語に対して類語を出力させ，その是非を評価しなさい．
1. Word2Vecのパラメタを変化させ，出力の違いを観察しなさい．
1. Word2Vecに与える学習データについて，GinZaやMeCab，ならびに扱う品詞を調整することで出力にどのような差異が生じるかを調べなさい．

### 参考文献
1. Radim Řehůřek, GENSIM, https://radimrehurek.com/gensim/
1. PyPI, gensim 4.3.2, https://pypi.org/project/gensim/
1. @singletongue, WikiEntVec, https://github.com/singletongue/WikiEntVec/releases
1. 鈴木正敏, 日本語 Wikipedia エンティティベクトル, https://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/
1. @taku910, MeCab: Yet Another Part-of-Speech and Morphological Analyzer, https://taku910.github.io/mecab/
1. @polm, fugashi, https://github.com/polm/fugashi
1. PyPI, fugashi 1.3.0, https://pypi.org/project/fugashi/
1. PyPi, unidic 1.1.0, https://pypi.org/project/unidic/
1. 国立国語研究所, UniDic, https://clrd.ninjal.ac.jp/unidic/download.html
1. 青空文庫, 青空文庫, https://www.aozora.gr.jp/

